<a href="https://colab.research.google.com/github/gorovuha/text_generation/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import gensim
from gensim.models import Word2Vec

model_21 = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/220/model.bin', binary = True)
model_19 = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/kursovaya/model.bin', binary = True)
my_model = Word2Vec.load('word2vec.model')

In [38]:
word = 'вход_NOUN'
print(model_21.most_similar(positive=word, topn=5))
print(model_19.most_similar(positive=word, topn=5))
print(my_model.wv.most_similar(positive=word, topn=5))

[('входной_ADJ', 0.6653426885604858), ('проход_NOUN', 0.6625036001205444), ('вестибюль_NOUN', 0.6128919124603271), ('лестница_NOUN', 0.5827667713165283), ('вход_PROPN', 0.5818999409675598)]
[('проход_NOUN', 0.6147749423980713), ('вход_PROPN', 0.6047937273979187), ('вестибюль_NOUN', 0.5664801597595215), ('арка_NOUN', 0.5368366837501526), ('входной_ADJ', 0.5250890254974365)]
[('подъезд_NOUN', 0.5856289863586426), ('въезд_NOUN', 0.5836901068687439), ('дверь_NOUN', 0.5815518498420715), ('ворота_NOUN', 0.5629271268844604), ('ворот_NOUN', 0.5618767738342285)]


In [ ]:
!pip install ufal.udpipe
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.9/936.9 kB 50.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=2aa0fc1514395925240f5e51f2893b31ff2e3d2279b1a9144fbde6330809a1eb
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [ ]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
   entities = {'PROPN'}
   named = False  # переменная для запоминания того, что нам встретилось имя собственное
   memory = []
   mem_case = None
   mem_number = None
   tagged_propn = []

   # обрабатываем текст, получаем результат в формате conllu:
   processed = pipeline.process(text)

   # пропускаем строки со служебной информацией:
   content = [l for l in processed.split('\n') if not l.startswith('#')]

   # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
   tagged = [w.split('\t') for w in content if w]

   for t in tagged:
       if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
           continue
       (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t 
       if not lemma or not token: # если слово пустое — пропускаем
           continue
       if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
           if '|' not in feats:
               tagged_propn.append('%s_%s' % (lemma, pos))
               continue
           morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
           if 'Case' not in morph or 'Number' not in morph:
               tagged_propn.append('%s_%s' % (lemma, pos))
               continue
           if not named:
               named = True
               mem_case = morph['Case']
               mem_number = morph['Number']
           if morph['Case'] == mem_case and morph['Number'] == mem_number:
               memory.append(lemma)
               if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                   named = False
                   past_lemma = '::'.join(memory)
                   memory = []
                   tagged_propn.append(past_lemma + '_PROPN ')
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s' % (lemma, pos))
       else:
           if not named:
               #if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                #   lemma = num_replace(token)
               tagged_propn.append('%s_%s' % (lemma, pos))
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s' % (lemma, pos))

   if not keep_punct: # обрабатываем случай, когда пользователь попросил не сохранять пунктуацию (по умолчанию она сохраняется)
       tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
   if not keep_pos:
       tagged_propn = [word.split('_')[0] for word in tagged_propn]
   return tagged_propn

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re

udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(modelfile):
       print('UDPipe model not found. Downloading...', file=sys.stderr)
       wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):

   print('Processing input...', file=sys.stderr)
   lines = text.split('\n')
   tagged = []
   for line in lines:
       # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
       output = process(process_pipeline, text=line)
       tagged_line = ' '.join(output)
       tagged.append(tagged_line)
   return '\n'.join(tagged)

c = 0
for filename in os.listdir("/content/drive/MyDrive/kursovaya/for_modeling"):
  text = open(os.path.join("/content/drive/MyDrive/kursovaya/for_modeling", filename), 'r', encoding='utf-8').read()
  split_regex = re.compile(r'[.|!|?|…]')
  sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(text)])
  sentenced_text = ''
  for s in sentences:
      sentenced_text += s + '\n'
#print(sentenced_text[0:350])
  processed_text = tag_ud(text=sentenced_text, modelfile=modelfile)
#print(processed_text[:350])
  with open('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)
    c+=1
    print(c)


Loading the model...
Processing input...


1


Processing input...


2


Processing input...


...

612


Processing input...


613


Processing input...


614


In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
f = 'my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

In [ ]:
model = gensim.models.Word2Vec(data, vector_size=500, window=3, min_count=2, sg=0)

In [ ]:
print(len(model.wv.key_to_index))

2625


In [ ]:
import os
from os import cpu_count
from gensim.models import Word2Vec
import gensim
import logging
import time

start = time.time()
data = []
c=0
for c in range(614):
    with open(os.path.join('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt')):
        data.extend(gensim.models.word2vec.LineSentence('/content/drive/MyDrive/kursovaya/for_training/my_text' + str(c) + '.txt'))
        c+=1
        print(c)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model = Word2Vec(data, min_count=1, window=3, vector_size=400, negative=10, alpha=0.03, min_alpha=0.0007, sg=0, workers=cpu_count())
w2v_model.save("word2vec.model")



print(w2v_model.wv.most_similar(positive='солнце_NOUN', topn=5))

print(len(w2v_model.wv.key_to_index))

print(time.time()-start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
